# 🧪 WFGY Diagnostic Notebook (v0.1)  
**Minimal sandbox to measure ΔS and spot common reasoning failures (Problem Map v1.0).**  
No API key required — runs locally in Colab using open-source sentence embeddings.

## Quick Start
1. ▶️ Run *all* cells (takes ~30 s on Colab free tier).  
2. Paste your **prompt / context** and the **LLM answer** when asked.  
3. Get ΔS, a basic failure classification, and a suggested WFGY fix.

> **Disclaimer**: This is a minimal demo. Full WFGY uses ΔS + λ_observe + E_resonance and 16 failure modes. 

In [ ]:
!pip install -q sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util
import math, textwrap

In [ ]:
# lightweight model for quick demo
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
def delta_s(a_vec, b_vec):
    """ΔS = 1 – cosine_similarity."""
    cos_sim = util.cos_sim(a_vec, b_vec).item()
    return 1 - cos_sim

def classify_failure(dS):
    """Very simple rule-based mapping (extend as needed)."""
    if dS > 0.80:
        return ('No.1  Hallucination / Chunk Drift', 'Use BBMC + boundary check')
    if dS > 0.60:
        return ('No.2  Interpretation Collapse', 'Insert λ_observe checkpoint')
    if dS > 0.40:
        return ('No.6  Logic Collapse', 'Trigger BBCR fallback')
    if dS > 0.25:
        return ('No.9  Entropy Collapse', 'Apply BBAM attention modulation')
    return ('Looks stable (ΔS ≤ 0.25)', 'No action — pass')

In [ ]:
prompt = input("Paste your prompt / context →\n\n")
answer = input("Paste the LLM answer →\n\n")

In [ ]:
# encode
p_vec = model.encode(prompt, convert_to_tensor=True)
a_vec = model.encode(answer, convert_to_tensor=True)

# compute ΔS & classify
dS = delta_s(p_vec, a_vec)
failure, fix = classify_failure(dS)

print("\n================  RESULTS  ================")
print(f"ΔS (semantic tension) : {dS:.3f}")
print(f"Detected              : {failure}")
print(f"Suggested fix         : {fix}")

---
### Next Steps  
• For the full 16-type Problem Map, see the main README.  
• To understand ΔS, λ_observe, E_resonance, check the WFGY 1.0 paper.  
• Feel free to fork this notebook and extend the rules or plug in your own embedding model.  

> 📎 Repo: https://github.com/onestardao/WFGY